In [1]:
from __future__ import print_function
import glob
import clip
from itertools import chain
import os
import cv2
import random
# import zipfile
import os.path as osp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
# from functools import reduce
import torch.nn as nn
# from einops import rearrange, repeat
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from PIL import Image
from imgaug import augmenters as iaa
# from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
# from skimage import io, img_as_float
import timm
from torchvision.transforms import ToPILImage
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import csv
import numpy as np
from sklearn.metrics import average_precision_score, accuracy_score, roc_auc_score, precision_recall_curve, f1_score
from scipy.ndimage import gaussian_filter

In [2]:
# Local imports
from utils import class_labels, get_dataset, get_transforms, DeepFakeSet, get_class_based_training_data
from models import clipmodel, dinov2, CLIPModelOhja
from trainer import train_model

In [3]:
device = 'cuda'

In [4]:
seed = 17
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

# Evaluate

In [9]:
model_ours = clipmodel()
model_ours.load_state_dict(torch.load('weights/CLIP_linear_prob_2_epoch_100k_vit_large_with_augs.pth'), strict=True)
model_ours.eval()
model_ours.cuda()
print('==')

CLIP OpenAI state dict loaded!
==


In [11]:
def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)

In [20]:
import json
def update_and_save_evaluation(model_name, dataset_name, accuracy, f1_score, average_precision):
    global model_evaluations
    
    # Check if the model key exists in the dictionary
    if model_name not in model_evaluations:
        model_evaluations[model_name] = {}
    
    # Check if the dataset key exists for the given model
    if dataset_name not in model_evaluations[model_name]:
        model_evaluations[model_name][dataset_name] = {}
    
    # Update evaluation results
    model_evaluations[model_name][dataset_name]["accuracy"] = accuracy
    model_evaluations[model_name][dataset_name]["f1_score"] = f1_score
    model_evaluations[model_name][dataset_name]["average_precision"] = average_precision

    save_file_name = model_name.split('/')[-1].split('.')[0]+'.json'
    
    # Save the updated dictionary to a JSON file
    with open(save_file_name, "w") as json_file:
        json.dump(model_evaluations, json_file, indent=2)

In [36]:
dataset_names = ['progan', 'biggan', 'cyclegan', 'eg3d', 'gaugan',  'stargan', 'stylegan', 'stylegan2', 'stylegan3', 
                 'dalle2', 'glide_50_27', 'glide_100_10', 'glide_100_27', 'guided', 'ldm_100', 'ldm_200', 'ldm_200_cfg',
                 'sd_512x512', 'sdxl', 'taming', 'deepfake', 'firefly', 'midjourney_v5', 'dalle3', 'faceswap']

model_names = ['weights/CLIP_linear_prob_2_epoch_10k_vit_large_with_augs.pth', 'weights/CLIP_linear_prob_2_epoch_20k_vit_large_with_augs.pth',
              'weights/CLIP_linear_prob_2_epoch_30k_vit_large_with_augs.pth', 'weights/CLIP_linear_prob_2_epoch_40k_vit_large_with_augs.pth']

model_names = ['weights/CLIP_linear_prob_2_epoch_100k_vit_large_with_augs.pth']

In [38]:
dataset_path = "Datasets/ICMRDataset/test/deepfake_eval/"
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.gif', '*.bmp', '*.tiff', '*.tif']  # Add more extensions as needed

In [45]:
sig = 2.0
model_evaluations = {}
tfms = transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
        ])

for model_name in model_names:
    model_ours = clipmodel()
    model_ours.load_state_dict(torch.load(model_name), strict=True)
    model_ours.eval()
    model_ours.cuda()
    print('==Loaded ' + model_name.split('/')[-1].split('.')[0] + '==')
    for dataset in dataset_names:
        print('Evaluating on: ' + dataset)
        labels_map = ["real", "fake"]
        count = 0
        all_images = []
        predicted_labels = []
        true_labels = []
        predicted_probs = []

        real_image_directory = dataset_path + dataset + '/images/val/n01440764/'
        fake_image_directory = dataset_path + dataset + '/images/val/n01443537/'
        
        images = []
        for extension in image_extensions:
            images.extend(glob.glob(os.path.join(real_image_directory, extension)))
            images.extend(glob.glob(os.path.join(fake_image_directory, extension)))
        images = sorted(images)
        images = [path.replace('\\','/') for path in images]
        print('Num. Images: ', len(images))
        y_pred = []
        for image in images:
            img = cv2.imread(image)
            # img = cv2_jpg(img, 50)
            # image = add_noise(image, 0.4)
            # gaussian_blur(img, sig)
            img = Image.fromarray(img)
            img = img.convert('RGB')
            # img = Image.open(image)
            img = tfms(img)
            y_true = []
            with torch.no_grad():
                with torch.cuda.amp.autocast():
                    outputs = model_ours(img.unsqueeze(0).to(device))
                    y_pred.extend(outputs.sigmoid().flatten().tolist())
                    # y_true.extend(label.flatten().tolist())
    
            for idx in torch.topk(outputs[0], k=1).indices.tolist():
                prob = torch.softmax(outputs[0], 0)[idx].item()
                if labels_map[idx] == 'real':
                    # print("real")
                    predicted_labels.append(0)
                    predicted_probs.append(1 - prob)
                else:
                    # print("fake")
                    predicted_labels.append(1)
                    predicted_probs.append(prob)
            
            # print(image.split('/')[-2])
            if 'n01440764' in image.split('/')[-2]:
                true_labels.append(0)
            else:
                true_labels.append(1)
        
        average_precision = 100 * average_precision_score(true_labels, predicted_probs)
        accuracy = 100 * accuracy_score(true_labels, predicted_labels)
        macro_f1 = 100.0 * f1_score(true_labels, predicted_labels, average="macro")
        
        update_and_save_evaluation(model_name, dataset, accuracy, macro_f1, average_precision)
        print('--------------')
        
    torch.cuda.empty_cache()

In [69]:
average_precision_score(true_labels, predicted_probs)

0.8296772251656449

In [70]:
accuracy_score(true_labels, predicted_labels)

0.6784655300222387

# Full Evaluation

In [12]:
model = CLIPModelOhja()
model.fc.load_state_dict(torch.load('weights/fc_weights.pth'), strict=True)
model.eval()
model.cuda()


CLIPModelOhja(
  (model): CLIP(
    (visual): VisionTransformer(
      (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): Sequential(
          (0): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
            )
            (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
              (gelu): QuickGELU()
              (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
            )
            (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          )
          (1): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamically

In [13]:
dataset_names = ['progan', 'biggan', 'cyclegan', 'eg3d', 'gaugan',  'stargan', 'stylegan', 'stylegan2', 'stylegan3', 
                 'dalle2', 'glide_50_27', 'glide_100_10', 'glide_100_27', 'guided', 'ldm_100', 'ldm_200', 'ldm_200_cfg',
                 'sd_512x512', 'sdxl', 'taming', 'deepfake', 'firefly', 'midjourney_v5', 'dalle3']
dataset_names = ['faceswap']
                #  'eg3d', 'stylegan3'] 'dalle2', 'deepfake', 'glide_50_27', 'glide_100_10', 'glide_100_27',
                # 'guided', 'ldm_100', 'ldm_200', 'ldm_200_cfg',
# dataset_names = ['firefly', 'midjourney_v5', 'dalle3']

In [14]:
import json

def update_and_save_evaluation(model_name, dataset_name, accuracy, f1_score, average_precision):
    global model_evaluations
    
    # Check if the model key exists in the dictionary
    if model_name not in model_evaluations:
        model_evaluations[model_name] = {}
    
    # Check if the dataset key exists for the given model
    if dataset_name not in model_evaluations[model_name]:
        model_evaluations[model_name][dataset_name] = {}
    
    # Update evaluation results
    model_evaluations[model_name][dataset_name]["accuracy"] = accuracy
    model_evaluations[model_name][dataset_name]["f1_score"] = f1_score
    model_evaluations[model_name][dataset_name]["average_precision"] = average_precision
    
    # Save the updated dictionary to a JSON file
    with open("ojha_model_evaluations_FACESWAP.json", "w") as json_file:
        json.dump(model_evaluations, json_file, indent=2)

In [15]:
model_evaluations = {}
tfms = transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
        ])

for dataset in dataset_names:
    labels_map = ["real", "fake"]
    count = 0
    all_images = []
    predicted_labels = []
    true_labels = []
    predicted_probs = []
    
    images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/'+dataset+'/images/val/n01440764/'+'*.jpg'))
    fake_images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/'+dataset+'/images/val/n01443537/'+'*.jpg'))

    if len(images) == 0:
        images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/'+dataset+'/images/val/n01440764/'+'*.png'))
    if len(fake_images) == 0:
        fake_images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/'+dataset+'/images/val/n01443537/'+'*.png'))
            
    # images = sorted(glob.glob('../CoOp/data/ImageNet/images/val/n01440764/*.jpg'))
    # fake_images = sorted(glob.glob('../CoOp/data/ImageNet/images/val/n01443537/*.png'))
    
    print(len(images))
    print(len(fake_images))
    
    # np.random.shuffle(fake_images)
    # np.random.shuffle(fake_images)
    
    all_images = images
    all_images.extend(fake_images)
    
    all_images = [path.replace('\\','/') for path in all_images]
    y_pred = []
    for image in all_images:
        img = cv2.imread(image)
        # img = cv2_jpg(img, 50)
        # image = add_noise(image, 0.4)
        img = Image.fromarray(img)
        img = img.convert('RGB')
        # img = Image.open(image)
        img = tfms(img)
        y_true = []
        with torch.no_grad():
            with torch.cuda.amp.autocast():
                outputs = model(img.unsqueeze(0).to(device))
                y_pred.extend(outputs.sigmoid().flatten().tolist())
                # y_true.extend(label.flatten().tolist())
    
        # print(outputs.sigmoid().flatten())
        if outputs.sigmoid().flatten() < 0.5:
            # print("real")
            predicted_labels.append(0)
            predicted_probs.append(outputs.sigmoid().flatten().cpu().numpy())
        else:
            # print("fake")
            predicted_labels.append(1)
            predicted_probs.append(outputs.sigmoid().flatten().cpu().numpy())
        
        # print(image.split('/')[-2])
        if 'n01440764' in image.split('/')[-2]:
            true_labels.append(0)
        else:
            true_labels.append(1)
        
    
    # y_true, y_pred = np.array(true_labels), np.array(y_pred)
    # train with compression and blur
    average_precision = 100 * average_precision_score(true_labels, predicted_probs)
    accuracy = 100 * accuracy_score(true_labels, predicted_labels)
    macro_f1 = 100.0 * f1_score(true_labels, predicted_labels, average="macro")
    
    update_and_save_evaluation('OjhaCLIP', dataset, accuracy, macro_f1, average_precision)
    print('--------------')

2800
2800
--------------


In [11]:
average_precision

95.81490083543112

In [10]:
accuracy

83.39999999999999

# Full Evaluation Wang et. el.

In [21]:
from networks.resnet import resnet50

In [22]:
dataset_names = ['progan', 'biggan', 'cyclegan', 'eg3d', 'gaugan',  'stargan', 'stylegan', 'stylegan2', 'stylegan3', 
                 'dalle2', 'glide_50_27', 'glide_100_10', 'glide_100_27', 'guided', 'ldm_100', 'ldm_200', 'ldm_200_cfg',
                 'sd_512x512', 'sdxl', 'taming', 'deepfake', 'firefly', 'midjourney_v5', 'dalle3']
dataset_names = ['faceswap']
                #  'eg3d', 'stylegan3'] 'dalle2', 'deepfake', 'glide_50_27', 'glide_100_10', 'glide_100_27',
                # 'guided', 'ldm_100', 'ldm_200', 'ldm_200_cfg',
# dataset_names = ['firefly', 'midjourney_v5', 'dalle3']

In [23]:
import json

def update_and_save_evaluation(model_name, dataset_name, accuracy, f1_score, average_precision):
    global model_evaluations
    
    # Check if the model key exists in the dictionary
    if model_name not in model_evaluations:
        model_evaluations[model_name] = {}
    
    # Check if the dataset key exists for the given model
    if dataset_name not in model_evaluations[model_name]:
        model_evaluations[model_name][dataset_name] = {}
    
    # Update evaluation results
    model_evaluations[model_name][dataset_name]["accuracy"] = accuracy
    model_evaluations[model_name][dataset_name]["f1_score"] = f1_score
    model_evaluations[model_name][dataset_name]["average_precision"] = average_precision
    
    # Save the updated dictionary to a JSON file
    with open("wang_model_evaluations_FACESWAP.json", "w") as json_file:
        json.dump(model_evaluations, json_file, indent=2)

In [24]:
model = resnet50(num_classes=1)
state_dict = torch.load('weights/blur_jpg_prob0.5.pth', map_location='cpu')
model.load_state_dict(state_dict['model'])
model.cuda()
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [25]:
model_evaluations = {}
tfms = transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

for dataset in dataset_names:
    labels_map = ["real", "fake"]
    count = 0
    all_images = []
    predicted_labels = []
    true_labels = []
    predicted_probs = []
    
    images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/'+dataset+'/images/val/n01440764/'+'*.jpg'))
    fake_images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/'+dataset+'/images/val/n01443537/'+'*.jpg'))

    if len(images) == 0:
        images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/'+dataset+'/images/val/n01440764/'+'*.png'))
    if len(fake_images) == 0:
        fake_images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/'+dataset+'/images/val/n01443537/'+'*.png'))
            
    # images = sorted(glob.glob('../CoOp/data/ImageNet/images/val/n01440764/*.jpg'))
    # fake_images = sorted(glob.glob('../CoOp/data/ImageNet/images/val/n01443537/*.png'))
    
    print(len(images))
    print(len(fake_images))
    
    # np.random.shuffle(fake_images)
    # np.random.shuffle(fake_images)
    
    all_images = images
    all_images.extend(fake_images)
    
    all_images = [path.replace('\\','/') for path in all_images]
    y_pred = []
    for image in all_images:
        img = cv2.imread(image)
        # img = cv2_jpg(img, 50)
        # image = add_noise(image, 0.4)
        img = Image.fromarray(img)
        img = img.convert('RGB')
        # img = Image.open(image)
        img = tfms(img)
        y_true = []
        with torch.no_grad():
            with torch.cuda.amp.autocast():
                outputs = model(img.unsqueeze(0).to(device))
                y_pred.extend(outputs.sigmoid().flatten().tolist())
                # y_true.extend(label.flatten().tolist())
    
        # print(outputs.sigmoid().flatten())
        if outputs.sigmoid().flatten() < 0.5:
            # print("real")
            predicted_labels.append(0)
            predicted_probs.append(outputs.sigmoid().flatten().cpu().numpy())
        else:
            # print("fake")
            predicted_labels.append(1)
            predicted_probs.append(outputs.sigmoid().flatten().cpu().numpy())
        
        # print(image.split('/')[-2])
        if 'n01440764' in image.split('/')[-2]:
            true_labels.append(0)
        else:
            true_labels.append(1)
    
    # y_true, y_pred = np.array(true_labels), np.array(y_pred)
    # train with compression and blur
    average_precision = 100 * average_precision_score(true_labels, predicted_probs)
    accuracy = 100 * accuracy_score(true_labels, predicted_labels)
    macro_f1 = 100.0 * f1_score(true_labels, predicted_labels, average="macro")
    
    update_and_save_evaluation('WangResNet0.5', dataset, accuracy, macro_f1, average_precision)
    print('--------------')

2800
2800
--------------


In [15]:
model = resnet50(num_classes=1)
state_dict = torch.load('weights/blur_jpg_prob0.1.pth', map_location='cpu')
model.load_state_dict(state_dict['model'])
model.cuda()
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
# model_evaluations = {}
tfms = transforms.Compose([
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

for dataset in dataset_names:
    labels_map = ["real", "fake"]
    count = 0
    all_images = []
    predicted_labels = []
    true_labels = []
    predicted_probs = []
    
    images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/'+dataset+'/images/val/n01440764/'+'*.jpg'))
    fake_images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/'+dataset+'/images/val/n01443537/'+'*.jpg'))

    if len(images) == 0:
        images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/'+dataset+'/images/val/n01440764/'+'*.png'))
    if len(fake_images) == 0:
        fake_images = sorted(glob.glob('../Datasets/ICMRDataset/test/deepfake_eval/'+dataset+'/images/val/n01443537/'+'*.png'))
            
    # images = sorted(glob.glob('../CoOp/data/ImageNet/images/val/n01440764/*.jpg'))
    # fake_images = sorted(glob.glob('../CoOp/data/ImageNet/images/val/n01443537/*.png'))
    
    print(len(images))
    print(len(fake_images))
    
    # np.random.shuffle(fake_images)
    # np.random.shuffle(fake_images)
    
    all_images = images
    all_images.extend(fake_images)
    
    all_images = [path.replace('\\','/') for path in all_images]
    y_pred = []
    for image in all_images:
        img = cv2.imread(image)
        # img = cv2_jpg(img, 50)
        # image = add_noise(image, 0.4)
        img = Image.fromarray(img)
        img = img.convert('RGB')
        # img = Image.open(image)
        img = tfms(img)
        y_true = []
        with torch.no_grad():
            with torch.cuda.amp.autocast():
                outputs = model(img.unsqueeze(0).to(device))
                y_pred.extend(outputs.sigmoid().flatten().tolist())
                # y_true.extend(label.flatten().tolist())
    
        # print(outputs.sigmoid().flatten())
        if outputs.sigmoid().flatten() < 0.5:
            # print("real")
            predicted_labels.append(0)
            predicted_probs.append(outputs.sigmoid().flatten().cpu().numpy())
        else:
            # print("fake")
            predicted_labels.append(1)
            predicted_probs.append(outputs.sigmoid().flatten().cpu().numpy())
        
        # print(image.split('/')[-2])
        if 'n01440764' in image.split('/')[-2]:
            true_labels.append(0)
        else:
            true_labels.append(1)
    
    # y_true, y_pred = np.array(true_labels), np.array(y_pred)
    # train with compression and blur
    average_precision = 100 * average_precision_score(true_labels, predicted_probs)
    accuracy = 100 * accuracy_score(true_labels, predicted_labels)
    macro_f1 = 100.0 * f1_score(true_labels, predicted_labels, average="macro")
    
    update_and_save_evaluation('WangResNet0.1', dataset, accuracy, macro_f1, average_precision)
    print('--------------')

4000
4000
--------------
2000
2000
--------------
1000
1000
--------------
1000
1000
--------------
5000
5000
--------------
1999
1999
--------------
1000
1000
--------------
1000
1000
--------------
950
950
--------------
1000
1000
--------------
1000
1000
--------------
1000
1000
--------------
1000
1000
--------------
1000
1000
--------------
1000
1000
--------------
1000
1000
--------------
1000
1000
--------------
1000
1000
--------------
1000
1000
--------------
1000
1000
--------------
2698
2698
--------------
1000
1000
--------------
1000
1000
--------------
1000
1000
--------------


In [ ]:
python test.py --gpu_ids 0 --which_epoch best_val --partition test --dataset_name celebahq-pgan-pretrained --real_im_path Datasets/ICMRDataset/test/deepfake_eval/'+biggan+'/images/val/n01440764 --fake_im_path Datasets/ICMRDataset/test/deepfake_eval/'+biggan+'/images/val/n01443537 --train_config checkpoints/gp1-gan-winversion_seed0_xception_block2_constant_p20/opt.yml


In [ ]:
python test_runs.py checkpoints/gp1-gan-winversion_seed0_xception_block2_constant_p20 gen_models test